In [51]:
import numpy as np

In [53]:
file_path = r'D:\Atomsk\las_cif\Al1Li1O6Si2_ICSD_c2_c_t0.xyz'
save_path = r'D:\MD_file\Corning\las2o6\1012_cube\lasp_inp.xyz'
save_region = r'D:\MD_file\Corning\las2o6\1012_cube\lasp_inp.region'

In [55]:
# 参数
interval_dis = 1.0


# 晶体

In [58]:
# 读取xyz文件并按照给定条件筛选坐标
def read_xyz_file_with_conditions(filename):
    coordinates = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines[2:]:  # 从第三行开始读取坐标数据
            parts = line.split()
            if len(parts) >= 4:
                atom = parts[0]
                #  转换一下单位 
                x = float(parts[1])
                y = float(parts[2])
                z = float(parts[3])
                coordinates.append((atom, x, y, z))
    return coordinates

# 使用示例
filename = file_path  # 替换为你的xyz文件名
cry_xyz = read_xyz_file_with_conditions(filename)

In [60]:
# 原始坐标列表
coordinates = cry_xyz
# 晶体中原子的计数
num_cry_dic = create_dic = {'O':0,'Si':0,'Al':0,'Li':0}
# 平移的距离
a, b, c = 0, 0, 0.0  # 替换为你需要的平移值
# 初始化最大最小值
max_x = max_y = max_z = float('-inf')
min_x = min_y = min_z = float('inf')

new_cry_xyz = []

# 平移坐标并寻找最大最小值
for atom, x, y, z in coordinates:
    # 对每个坐标平移
    new_x = x + a
    new_y = y + b
    new_z = z + c
    new_cry_xyz.append([atom,new_x,new_y,new_z])
    # 对晶体在的原子计数
    num_cry_dic[atom] += 1
    # 更新最大值
    if new_x > max_x:
        max_x = new_x
    if new_y > max_y:
        max_y = new_y
    if new_z > max_z:
        max_z = new_z
    
    # 更新最小值
    if new_x < min_x:
        min_x = new_x
    if new_y < min_y:
        min_y = new_y
    if new_z < min_z:
        min_z = new_z

# 输出平移后所有原子中最大的和最小的 x, y, z 值
print("晶体的初始xyz范围：")
print(f"x 值: {min_x} -- {max_x}")
print(f"y 值: {min_y} -- {max_y}")
print(f"z 值: {min_z} -- {max_z}")
print("晶体的原子数：")
print(len(cry_xyz))
print(num_cry_dic)

晶体的初始xyz范围：
x 值: -0.698745 -- 60.110055
y 值: 17.240722 -- 78.381319
z 值: -0.998862 -- 50.346257
晶体的原子数：
18980
{'O': 11721, 'Si': 3683, 'Al': 1819, 'Li': 1757}


In [62]:
is_charge = 1
charge_balance_dic = {'O':0,'Si':0,'Al':0,'Li':0}
sum_charge = 0
charge_dic = {'O':-1.2,'Si':2.4,'Al':1.8,'Li':0.6}
for i in num_cry_dic:
    sum_charge += (num_cry_dic[i] + charge_balance_dic[i] )*charge_dic[i]
print('总电荷数： {}'.format(round(sum_charge,1)))

总电荷数： -897.6


# 当遇到xyz小于0的，先全部给它加上 

In [37]:
# 只有确实是有小于O的部分才会执行 或者取值范围不为0-x的方向
if min_x!=0 or min_y!=0 or min_z!=0:
    # 原始坐标列表
    coordinates = cry_xyz
    # 平移的距离
    a, b, c = 0,0,0
    if min_x!=0:
        a=-min_x
    if min_y!=0:
        b = -min_y
    if min_z!=0:
        c = -min_z
    # 替换为你需要的平移值
    # 初始化最大最小值
    max_x = max_y = max_z = float('-inf')
    min_x = min_y = min_z = float('inf')
    
    new_cry_xyz = []
    
    # 平移坐标并寻找最大最小值
    for atom, x, y, z in coordinates:
        # 对每个坐标平移
        new_x = x + a
        new_y = y + b
        new_z = z + c
        new_cry_xyz.append([atom,new_x,new_y,new_z])
        # 更新最大值
        if new_x > max_x:
            max_x = new_x
        if new_y > max_y:
            max_y = new_y
        if new_z > max_z:
            max_z = new_z
        
        # 更新最小值
        if new_x < min_x:
            min_x = new_x
        if new_y < min_y:
            min_y = new_y
        if new_z < min_z:
            min_z = new_z

    min_a,min_b,min_c = min_x,min_y,min_z
    max_a,max_b,max_c = max_x,max_y,max_z
    # 输出平移后所有原子中最大的和最小的 x, y, z 值
    print("修正过后的晶体的xyz范围：")
    print(f"x 值: {min_a} -- {max_a}")
    print(f"y 值: {min_b} -- {max_b}")
    print(f"z 值: {min_c} -- {max_c}")

修正过后的晶体的xyz范围：
x 值: 0.0 -- 13.336793
y 值: 0.0 -- 10.217253
z 值: 0.0 -- 13.451135


In [10]:
glass_cube_len = 38
glass_volum = glass_cube_len**3
cry_volum = (max_x-min_x)*(max_y-min_y)*(max_z-min_z)

len_box = round(((glass_volum+cry_volum))**(1/3),1)
print('Cube_length: {}'.format(len_box))

Cube_length: 46.1


In [11]:
print(f"x 值: {min_x} -- {max_x}")
print(f"y 值: {min_y} -- {max_y}")
print(f"z 值: {min_z} -- {max_z}")

x 值: 0.0 -- 37.055113
y 值: 0.0 -- 30.537431
z 值: 0.0 -- 38.34574


In [12]:
box_size = [len_box,len_box,len_box]
low_x, high_x = round(0.5*len_box - (max_x-min_x)*0.5-interval_dis,1), round(0.5*len_box - (max_x-min_x)*0.5+max_x+ interval_dis,1)
low_y, high_y = round(0.5*len_box - (max_y-min_y)*0.5-interval_dis,1), round(0.5*len_box - (max_y-min_y)*0.5+max_y+ interval_dis,1)
low_z, high_z = round(0.5*len_box - (max_z-min_z)*0.5-interval_dis,1), round(0.5*len_box - (max_z-min_z)*0.5+max_z+ interval_dis,1)
cry_range = [[low_x,high_x],[low_y,high_y],[low_z,high_z]]
# 把两个list加到一起去 算一下总的原子数
cry_dic_list = []
for i in num_cry_dic:
    temp =num_cry_dic[i]
    cry_dic_list.append(temp)

glass_create_dic = {'O':2856,'Si':714,'Al':714,'Li':714}
print("盒子尺寸：")
print(box_size)
print("晶体区域：")
print(cry_range)
print("玻璃中的原子数量：")
print(glass_create_dic)
# 如果需要平衡电荷 =1
if is_charge==1:
    for i in glass_create_dic:
        glass_create_dic[i]+=charge_balance_dic[i]
    print("为中和电荷添加的原子数：")
    print(charge_balance_dic)
    print("平衡电荷后的玻璃原子数量：")
    print(glass_create_dic)

盒子尺寸：
[46.1, 46.1, 46.1]
晶体区域：
[[3.5, 42.6], [6.8, 39.3], [2.9, 43.2]]
玻璃中的原子数量：
{'O': 2856, 'Si': 714, 'Al': 714, 'Li': 714}
为中和电荷添加的原子数：
{'O': 0, 'Si': 72, 'Al': 80, 'Li': 82}
平衡电荷后的玻璃原子数量：
{'O': 2856, 'Si': 786, 'Al': 794, 'Li': 796}


In [13]:
# 平移晶体元素 xyz

# 原始坐标列表
coordinates = new_cry_xyz
# 平移的距离
a, b, c = 0.5*len_box - (max_x-min_x)*0.5,  0.5*len_box - (max_y-min_y)*0.5, 0.5*len_box - (max_z-min_z)*0.5  # 替换为你需要的平移值
# 初始化最大最小值
max_x = max_y = max_z = float('-inf')
min_x = min_y = min_z = float('inf')

final_cry_xyz = []

# 平移坐标并寻找最大最小值
for atom, x, y, z in coordinates:
    # 对每个坐标平移
    new_x = x + a
    new_y = y + b
    new_z = z + c
    final_cry_xyz.append([atom,new_x,new_y,new_z])
    # 更新最大值
    if new_x > max_x:
        max_x = new_x
    if new_y > max_y:
        max_y = new_y
    if new_z > max_z:
        max_z = new_z
    
    # 更新最小值
    if new_x < min_x:
        min_x = new_x
    if new_y < min_y:
        min_y = new_y
    if new_z < min_z:
        min_z = new_z

# 输出平移后所有原子中最大的和最小的 x, y, z 值
print(f"x 值: {min_x} -- {max_x}")
print(f"y 值: {min_y} -- {max_y}")
print(f"z 值: {min_z} -- {max_z}")
print(cry_range)

x 值: 4.522443500000001 -- 41.5775565
y 值: 7.7812845 -- 38.3187155
z 值: 3.877130000000001 -- 42.22287
[[3.5, 42.6], [6.8, 39.3], [2.9, 43.2]]


In [14]:
0.5*len_box - (max_y-min_y)*0.5

7.781284499999998

# 玻璃

In [16]:
import numpy as np
import random
total_atoms = sum(glass_create_dic.values())
# 函数判断位置是否在禁止区域内
def in_forbidden_range(x, y, z, cry_range):
    return (cry_range[0][0] <= x <= cry_range[0][1] and
            cry_range[1][0] <= y <= cry_range[1][1] and
            cry_range[2][0] <= z <= cry_range[2][1])
# 生成候选坐标
def generate_positions(box_size, cry_range, num_atoms):
    positions = []
    while len(positions) < num_atoms:
        # 随机生成一个坐标 (x, y, z)
        x = random.uniform(0, box_size[0])
        y = random.uniform(0, box_size[1])
        z = random.uniform(0, box_size[2])
        
        # 检查是否在禁止区域内
        if not in_forbidden_range(x, y, z, cry_range):
            positions.append([x, y, z])
    
    return positions

# 生成均匀分布的随机坐标
positions = generate_positions(box_size, cry_range, total_atoms)

# 随机分配原子种类
atom_types = []
for atom, count in glass_create_dic.items():
    atom_types += [atom] * count

# 打乱原子的顺序
random.shuffle(atom_types)

glass_xyz = []
temp_list = []
# 输出原子及其对应的位置
for i in range(total_atoms):
    atom = atom_types[i]
    x, y, z = positions[i]
    #print(f"{atom}: ({x:.2f}, {y:.2f}, {z:.2f})")
    temp = [atom,x,y,z]
    glass_xyz.append(temp)
    temp_list.append([x,y,z])

In [17]:
import numpy as np
from scipy.spatial.distance import cdist

# 盒子尺寸
box_size = np.array(box_size)

# 最小距离阈值
min_distance = 0.2  # 替换为你的实际最小距离阈值

# 计算周期性边界条件下的距离
def periodic_distance(p1, p2, box_size):
    diff = np.abs(p1 - p2)
    diff = np.minimum(diff, box_size - diff)  # 考虑周期性边界
    return np.sqrt(np.sum(diff**2))

# 检查原子是否距离过近
def check_closest_atoms(positions, box_size, min_distance):
    num_atoms = len(positions)
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = periodic_distance(positions[i], positions[j], box_size)
            if distance < min_distance:
                print(f"原子 {i} 和原子 {j} 之间的距离过近: {distance:.2f}")



# 原子位置（示例）
# 检查原子距离
temp_positions = np.array(temp_list)
check_closest_atoms(temp_positions, box_size, min_distance)


# 组合

In [20]:
all_xyz = glass_xyz +final_cry_xyz

In [21]:
print('atoms of crystal: ' + str(len(cry_xyz)))
print('atoms of glass: ' + str(len(glass_xyz)))
print('All atoms: ' + str(len(glass_xyz)+len(cry_xyz)))

atoms of crystal: 3981
atoms of glass: 5232
All atoms: 9213


check_list = []
for i in all_xyz:
    check_list.append(i[1:])

temp_positions = np.array(check_list)
check_closest_atoms(temp_positions, box_size, min_distance)

In [23]:
ele_dic = {'O':1,'Si':2,'Al':3,'Li':4}
# 将坐标写回到xyz文件中
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write(f"{len(coordinates)}\n")
        f.write("Atoms\n")
        for atom, x, y, z in coordinates:
            f.write(f"{ele_dic[atom]}    {x:.6f}    {y:.6f}    {z:.6f}\n")

# 使用示例
save_path = save_path  # 设置保存文件的路径和文件名
write_xyz_file(save_path, all_xyz)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\MD_file\Corning\las2o6\1012_cube\lasp_inp.xyz


In [24]:
# 将lmp文件写进las_112.region
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write("region box_region block {} {} {} {} {} {}\n\n".format(0, len_box, 0, len_box, 0, len_box))
        f.write("region cry_region block {} {} {} {} {} {}\n\n".format(low_x, high_x, low_y, high_y,low_z, high_z))
        f.write("group all_atoms region box_region\n\n")
        f.write("group cry_group region cry_region\n\n")
        f.write("group glass_group subtract all_atoms cry_group\n")
# 使用示例
save_path = save_region  # 设置保存文件的路径和文件名
write_xyz_file(save_path, all_xyz)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\MD_file\Corning\las2o6\1012_cube\lasp_inp.region


In [25]:
# 计算总电荷数量
print('crystal atoms: {}  {} '.format(len(final_cry_xyz),num_cry_dic))
print('glass atoms:   {}  {}'.format(len(glass_xyz),glass_create_dic))
print(charge_dic)
charge = 0
for i in num_cry_dic:
    charge += (num_cry_dic[i] + glass_create_dic[i]) * charge_dic[i]
print(charge)

crystal atoms: 3981  {'O': 2513, 'Si': 740, 'Al': 364, 'Li': 364} 
glass atoms:   5232  {'O': 2856, 'Si': 786, 'Al': 794, 'Li': 796}
{'O': -1.2, 'Si': 2.4, 'Al': 1.8, 'Li': 0.6}
0.0


In [26]:
# 计算总电荷数量
print('crystal atoms: {}  {} '.format(len(final_cry_xyz),num_cry_dic))
print('glass atoms:   {}  {}'.format(len(glass_xyz),glass_create_dic))
count_dic = {'O': 0, 'Si': 0, 'Al': 0, 'Li': 0} 
for i_line in glass_xyz:
    i_type = i_line[0]
    count_dic[i_type] += 1
for i_line in final_cry_xyz:
    i_type = i_line[0]
    count_dic[i_type] += 1
print(count_dic)
print(charge_dic)
charge = 0

for i in charge_dic:
    charge += charge_dic[i]*count_dic[i]
print(round(charge,5))

crystal atoms: 3981  {'O': 2513, 'Si': 740, 'Al': 364, 'Li': 364} 
glass atoms:   5232  {'O': 2856, 'Si': 786, 'Al': 794, 'Li': 796}
{'O': 5369, 'Si': 1526, 'Al': 1158, 'Li': 1160}
{'O': -1.2, 'Si': 2.4, 'Al': 1.8, 'Li': 0.6}
0.0


In [27]:
new_charge = 0
for i in num_cry_dic:
    new_charge += charge_dic[i]*count_dic[i]
print(round(new_charge,5))

0.0


In [28]:
glass_create_dic

{'O': 2856, 'Si': 786, 'Al': 794, 'Li': 796}

In [29]:
print("盒子尺寸：")
print(box_size)
print("晶体区域：")
print(cry_range)
print("玻璃中的原子数量：")
print(glass_create_dic)
print("晶体中的原子数量：")
print(num_cry_dic)


盒子尺寸：
[46.1 46.1 46.1]
晶体区域：
[[3.5, 42.6], [6.8, 39.3], [2.9, 43.2]]
玻璃中的原子数量：
{'O': 2856, 'Si': 786, 'Al': 794, 'Li': 796}
晶体中的原子数量：
{'O': 2513, 'Si': 740, 'Al': 364, 'Li': 364}
